In [2]:
%pip install pymysql

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pymysql

In [3]:
try:
    conn = pymysql.connect(
        host="localhost",
        user="hskim",
        password="153104",
        database="study",
        charset="utf8mb4"
    )
    print("연결 성공")
except:
    print("연결 실패")

연결 성공


In [16]:
def read_config(p:str)->dict:
    with open(p, 'r') as f:
        config=f.readlines()
    config_dict={}

    for l in config:
        idx =l.index('=')
        k=l[:idx] 
        v=l[idx+1:]
        config_dict[k]=v.rstrip()
    return config_dict